**Opti & Contrôle - TP1**

DESFORGES Guillaume & DUMONT Louis

In [ ]:
import numpy as np

In [ ]:
import matplotlib

In [ ]:
matplotlib.rcParams['figure.figsize'] = (15, 10)

In [ ]:
from src import datas_r

# Problème primal

On s'intéresse au problème primal d'optimisation **sans contraintes**.

## Résultats théoriques

In [ ]:
%%javascript
MathJax.Hub.Config({
    TeX: { equationNumbers: { autoNumber: "AMS" } }
});

Montrons les équivalences des différents problèmes:

On part du problème d'optimisation :

\begin{equation}
\underset{Aq-f = 0}{\underset{q\in \mathbb{R}^{n}}{min}< q, r\bullet q\bullet | q|> + <p, f_r>}
\end{equation}

La décomposition  de $A$ et $f$ sous la forme

$$A = \begin{pmatrix}
A_r \\ A_d
\end{pmatrix}
\quad
f= \begin{pmatrix}
f_r \\ f_d
\end{pmatrix}$$

Permet de réecrire la condition sous la forme:

$$\left\{
\begin{array}{l}
    A_d q - f_d = 0 \\
    A_r q - f_r = 0
\end{array}\right.$$

La deuxième de ces conditions s'intègre directement dans l'énergie en remplaçant $f_r$ par $A_r q$. On obtient alors le problème équivalent:

\begin{equation}
\underset{A_dq-f_d = 0}{\underset{q\in \mathbb{R}^{n}}{min}< q, r\bullet q\bullet | q|> + <p, A_rq>}
\end{equation}

De plus, la matrice $A_d$ est de plein rang $m_d$, ce qui permet de la réecrire $A_d = (A_{d,T}\; A_{d,C})$, où $A_{d,T}$ est inversible.

En écrivant de même $q = \begin{pmatrix} q_T \\ q_C \end{pmatrix}$, la condition de $(2)$ équivaut à $$q_T = A_{d,T}^{-1} (f_d - A_{d,C}q_C )$$

Une solution de $(2)$ s'écrit donc $q =q^{(0)}+B q_C$ avec 
$$q^{(0)} = \begin{pmatrix}A^{-1}_{d,T}f_d \\ 0_{n-m_d}\end{pmatrix}$$ et
$$B = \begin{pmatrix}-A^{-1}_{d,T}A_{d,C} \\ I_{n-m_d}\end{pmatrix}$$

et est solution de :

\begin{equation}
\underset{q_c \in \mathcal{R}^{n - m_d}}{min} \frac{1}{3}<q^{(0)} + B q_c, r \bullet(q^{(0)} + B q_c)\bullet | q^{(0)} + B q_c| > + <p_r, A_r(q^{(0)} + B q_c)>
\end{equation}

Réciproquement, une solution $q_C$ de $(3)$ permet de construire $q = q^{(0)} + B q_C$ solution de $(2)$ et les problèmes sont donc équivalents.

On calcule les formules du **gradient** et de la **hessienne**.

Pour le gradient on a l'expression suivante :

$\nabla_{q_c}(J) = B^T (r \bullet q \bullet |q| + A_r^T p_r)$

Et pour la Hessienne:

$H_{q_c}(J) = 2B^T (r \bullet |q| \bullet B^T)^T $

Où l'on a étendu la définition de $\bullet$ par : $v\bullet A$ est le produit matriciel de la matrice diagonale $diag(v_0 ,..., v_n)$ et de $X$.

## Oracle

On implémente un **oracle**, c'est à dire une fonction qui puise dans les données du problème pour donner à l'algorithme d'optimisation les informations qui lui sont nécessaire : valeur du critère, vecteur gradient du critère et si demandé matrice Hessienne du critère.

In [ ]:
from src.oracle import oracle

In [ ]:
x0 = np.random.normal(size=datas_r.n-datas_r.md)

In [ ]:
print("Test de l'oracle")
# test pour une valeur aléatoire
loss, gradient, hessian = oracle(x0, compute_hessian=True)
print("Critère :", loss)
print("Gradient :", gradient)
print("Hessienne :", hessian)

## Descentes de gradient à pas fixe ou à pas optimal

Dans un premier temps, on implémente la descente de gradient à pas fixe :

In [ ]:
from src.gradient import gradient

### Test

On teste la descente de gradient à pas fixe sur un problème trivial : $x^2$.

In [ ]:
# oracle test : la fonction carré en 1 dimension
oracle_test = lambda x: (x**2, np.array(x*2), None)
print("Descente de gradient à pas fixe :")
result_pas_fixe = gradient(oracle_test, np.array([2]), iter_max=20, default_gradient_step=0.1, threshold=1e-16, use_wolfe=False, verbose=True)

On constate une convergence lente pour un problème simple. Une solution est alors d'augmenter le pas du gradient, mais ce n'est pas suffisant pour un problème plus complexe. On utilise alors l'algorithme de Wolfe pour trouver le pas optimal.

In [ ]:
print("Descente de gradient à pas optimal :")
result_pas_opti = gradient(oracle_test, np.array([2]), iter_max=50, threshold=1e-16, verbose=True, visual=False)

Comme on pouvait s'y attendre, sur un problème convexe à une dimension la solution optimale est trouvée immédiatement.

### Utilisation sur le problème

On peut utiliser ces algorithmes sur notre problème :

In [ ]:
print("PAS FIXE :")
result = gradient(oracle, x0, default_gradient_step=0.0001, use_wolfe=False)

In [ ]:
print("PAS OPTIMAL :")
result = gradient(oracle, x0)

## Gradient conjugué

On implémente la méthode de gradient conjugué non linéaire avec l'algorithme de Polak-Ribière.

In [ ]:
from src.gradient_conjug_polak import gradient_polak

In [ ]:
result_polak = gradient_polak(oracle, x0)

## Méthode de Newton

On implémente la méthode de Newton.

In [ ]:
from src.newton import newton

In [ ]:
result_newton = newton(oracle, x0)

## Méthode quasi-Newton (BFGS)

On implémente la méthode dite "quasi-Newton" BFGS, qui utilise une approximation de l'inverse du hessien.

In [ ]:
from src.bfgs import bfgs

In [ ]:
result_bfgs = bfgs(oracle, x0)

# Problème dual

## Résultats théoriques

*TODO*

- Ecrire le lagrangien associé au problème sous contraintes (14) et ses conditions d’optimalité ;
- Vérifier que ces conditions correspondent aux équations (6) et (8) de l’équilibre du réseau.

- Constater que la minimisation en $q$ du lagrangien se fait de manière explicite ;
- Donner l’expression de l’argmin $q^♯$ en fonction du multiplicateur dual $\lambda$ et calculer les expressions de la fonction duale $\Phi$, de son gradient et de son hessien en fonction de $\lambda$.


Le Lagrangien du problème $(14)$ s'écrit:

$\mathbb{L}(q, \lambda) = \frac{1}{3} <q, r\bullet q\bullet | q|> + <p_r, A_rq> + \lambda^T(A_dq - f_d)$

On cherche dans un premier temps à le minimiser (à $\lambda$ fixé), c'est à dire à trouver les points d'annulation de son gradient. Cette condition s'écrit $r\bullet q \bullet |q| + A_r^T p_r + A_d^T \lambda = 0$ (on retrouve l'équation $(8)$ de l'énoncé)

L'extrémum de $\mathbb{L}$ est donc atteint en $\hat{q}(\lambda) = sign(X)\sqrt{|X|}$ où $X = -\frac{A^{T}_r p_r + A^{T}_d \lambda}{r}$

On veut maintenant maximiser par rapport à $\lambda$ le minimum que nous venons de calculer. Pour celà nous calculons son gradient: on note $H(\lambda) = \mathbb{L}(\hat{q}(\lambda),\lambda)$, et on a $\nabla H(\lambda) = A_d\hat{q}(\lambda) - f_d$.
La condition d'optimalité est donc $A_d\hat{q}(\lambda) - f_d = 0$, dans laquelle on retrouve l'équation $(6)$ de l'énoncé.

On dispose comme on l'a vu d'une expression explicite pour $\hat{q}(\lambda)$ minimisant le lagrangien à $\lambda$ fixé.
Pour résoudre le problème de maximisation (ou son opposé problème de minimisation) du minimum du lagrangien en fonction de $\lambda$ par les méthodes précédentes, il nous faut connaître le gradient et la hessienne de $H(\lambda)$.

Le gradient a déjà été présenté: $\nabla H(\lambda) = A_d\hat{q}(\lambda) - f_d$

La Hessienne est, quant à elle, donnée par $\nabla \hat{q}(\lambda) = A_d\quad D(\lambda) \quad A_d^T$, où $D(\lambda)=-\frac{1}{r\bullet \sqrt{|X|}}\bullet I_d$ où $I_d$ est la matrice identité de taille $n$.

Une fois ces deux expressions établies il suffit alors d'appliquer les algorithmes d'optimisation précédents à l'opposé du problème dual (on prend l'opposé pour passer d'un problème de maximisation à un problème de minimisation).



## Oracle

On implémente l'oracle avec le Lagrangien :

In [ ]:
from src.oracle_lagrange import oracle as oracle_dual

In [ ]:
u0 = np.random.normal(size=datas_r.md)

## Résolution

On résoud le problème dual grâce aux algorithmes précédents :

In [ ]:
print("PAS FIXE :")
result = gradient(oracle_dual, u0, default_gradient_step=0.2, use_wolfe=False, iter_max=10000)

In [ ]:
print("PAS OPTIMAL :")
result = gradient(oracle_dual, u0, threshold=0.01, iter_max=5000)

In [ ]:
result_polak = gradient_polak(oracle_dual, u0)

In [ ]:
result_newton = newton(oracle_dual, u0)

In [ ]:
result_bfgs = bfgs(oracle_dual, u0)